# Creating Custom Agents with AutoGen

This lesson explores how to create custom agents in AutoGen by implementing the `BaseChatAgent` class. We'll create two interesting examples: a Dice Rolling Agent and a Temperature Conversion Agent.

First, let's set up our environment:

In [4]:
import os
import getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [5]:
try:
    import nest_asyncio
    nest_asyncio.apply()
    print("Async environment configured for Jupyter.")
except ImportError:
    print("Please install nest_asyncio with `pip install nest_asyncio`")

Async environment configured for Jupyter.


In [6]:
import os
import getpass
from typing import AsyncGenerator, List, Sequence, Tuple
from autogen_agentchat.agents import BaseChatAgent
from autogen_agentchat.base import Response
from autogen_agentchat.messages import AgentEvent, ChatMessage, TextMessage
from autogen_core import CancellationToken

## 1. Creating a Dice Rolling Agent

Let's create a custom agent that simulates rolling different types of dice (d4, d6, d8, d20):

In [7]:
import random

class DiceRollingAgent(BaseChatAgent):
    def __init__(self, name: str):
        super().__init__(name, "An agent that simulates rolling different types of dice.")
        self.valid_dice = [4, 6, 8, 20]
        
    @property
    def produced_message_types(self) -> Sequence[type[ChatMessage]]:
        return (TextMessage,)
        
    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        response: Response | None = None
        async for message in self.on_messages_stream(messages, cancellation_token):
            if isinstance(message, Response):
                response = message
        assert response is not None
        return response
        
    async def on_messages_stream(
        self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken
    ) -> AsyncGenerator[AgentEvent | ChatMessage | Response, None]:
        inner_messages: List[AgentEvent | ChatMessage] = []
        
        for dice in self.valid_dice:
            result = random.randint(1, dice)
            msg = TextMessage(content=f"Rolling d{dice}: {result}", source=self.name)
            inner_messages.append(msg)
            yield msg
            
        yield Response(
            chat_message=TextMessage(content="All dice have been rolled!", source=self.name),
            inner_messages=inner_messages
        )
        
    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        pass

async def run_dice_agent():
    dice_agent = DiceRollingAgent("dice_master")
    async for message in dice_agent.on_messages_stream([], CancellationToken()):
        if isinstance(message, Response):
            print(message.chat_message.content)
        else:
            print(message.content)

In [12]:
# Use this when running in a script:
# asyncio.run(run_dice_agent())

await run_dice_agent()

Rolling d4: 1
Rolling d6: 5
Rolling d8: 4
Rolling d20: 7
All dice have been rolled!


## 2. Creating a Temperature Conversion Agent

Now let's create an agent that converts temperatures between Celsius and Fahrenheit:

In [13]:
class TemperatureAgent(BaseChatAgent):
    def __init__(self, name: str, description: str, convert_func):
        super().__init__(name, description)
        self._convert_func = convert_func
        self._message_history: List[ChatMessage] = []
        
    @property
    def produced_message_types(self) -> Sequence[type[ChatMessage]]:
        return (TextMessage,)
        
    async def on_messages(self, messages: Sequence[ChatMessage], cancellation_token: CancellationToken) -> Response:
        self._message_history.extend(messages)
        
        try:
            assert isinstance(self._message_history[-1], TextMessage)
            temp = float(self._message_history[-1].content)
            result = self._convert_func(temp)
            response_message = TextMessage(
                content=f"{temp:.1f} converts to {result:.1f}",
                source=self.name
            )
            self._message_history.append(response_message)
            return Response(chat_message=response_message)
        except (ValueError, AssertionError):
            error_message = TextMessage(
                content="Please provide a valid temperature number.",
                source=self.name
            )
            return Response(chat_message=error_message)
            
    async def on_reset(self, cancellation_token: CancellationToken) -> None:
        self._message_history = []

async def run_temperature_agents():
    # Create conversion agents
    to_celsius = TemperatureAgent(
        "celsius_converter",
        "Converts Fahrenheit to Celsius",
        lambda f: (f - 32) * 5/9
    )
    
    to_fahrenheit = TemperatureAgent(
        "fahrenheit_converter",
        "Converts Celsius to Fahrenheit",
        lambda c: (c * 9/5) + 32
    )
    
    # Test conversions
    test_temps = ["98.6", "37", "0", "100"]
    
    for temp in test_temps:
        msg = [TextMessage(content=temp, source="user")]
        response = await to_celsius.on_messages(msg, CancellationToken())
        print(f"F to C: {response.chat_message.content}")
        
        response = await to_fahrenheit.on_messages(msg, CancellationToken())
        print(f"C to F: {response.chat_message.content}")
        print()

# Use this when running in a script:
# asyncio.run(run_temperature_agents())

In [14]:
await run_temperature_agents()

F to C: 98.6 converts to 37.0
C to F: 98.6 converts to 209.5

F to C: 37.0 converts to 2.8
C to F: 37.0 converts to 98.6

F to C: 0.0 converts to -17.8
C to F: 0.0 converts to 32.0

F to C: 100.0 converts to 37.8
C to F: 100.0 converts to 212.0



These examples demonstrate how to create custom agents by implementing the required methods from `BaseChatAgent`:

1. `__init__`: Initialize the agent with a name and description
2. `produced_message_types`: Define what types of messages the agent can produce
3. `on_messages`: Handle incoming messages and generate responses
4. `on_messages_stream`: (Optional) Stream responses as they're generated
5. `on_reset`: Reset the agent's state

The DiceRollingAgent shows how to create a streaming response with multiple messages, while the TemperatureAgent demonstrates how to handle specific calculations and maintain message history.

To use these agents, you would typically run them in an async context. The example code includes commented-out `asyncio.run()` calls that you would use in a regular Python script. In a Jupyter notebook, you can use the await keyword directly as shown in the example functions.